# UDP Client Test Notebook

이 노트북은 `udp_client_test.py`를 기반으로, UDP 소켓 통신으로 서버에서 전송한 영상 프레임을 수신하는 과정을 단계별로 설명합니다.

## 1) UDP와 Socket 핵심 개념

- **Socket**: 네트워크 통신의 시작점(통신 엔드포인트)입니다.
- **UDP (User Datagram Protocol)**:
  - 연결 설정(Handshake) 없이 데이터를 전송합니다.
  - 속도가 빠르지만, 패킷 손실/순서 보장이 없습니다.
  - 실시간 스트리밍(영상, 센서 데이터)에 자주 사용됩니다.

이 예제에서는 서버가 보낸 Base64 인코딩 영상 데이터를 UDP로 받아 OpenCV로 디코딩/출력합니다.

## 2) 라이브러리 로드와 기본 설정

In [ ]:
import socket
import time
import base64

import cv2
import numpy as np

# UDP 수신 버퍼 크기
BUFF_SIZE = 65536

# 서버 IP/포트 (서버 실행 환경에 맞게 수정)
HOST_IP = "192.168.0.52"
PORT = 9999

print(f"HOST_IP={HOST_IP}, PORT={PORT}")

## 3) UDP 소켓 생성

- `AF_INET`: IPv4
- `SOCK_DGRAM`: UDP 타입 소켓
- `SO_RCVBUF`: 수신 버퍼 크기 설정

In [ ]:
client_socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
client_socket.setsockopt(socket.SOL_SOCKET, socket.SO_RCVBUF, BUFF_SIZE)

print("UDP client socket created")

## 4) 서버에 시작 메시지 전송

일반적으로 서버가 클라이언트 주소를 알 수 있도록 첫 패킷(`Hello`)을 보냅니다.

In [ ]:
message = b"Hello"
client_socket.sendto(message, (HOST_IP, PORT))
print("Start message sent to server")

## 5) 프레임 수신 및 화면 출력

흐름:
1. UDP 패킷 수신
2. Base64 디코딩
3. NumPy 버퍼 변환
4. OpenCV 이미지 디코딩
5. FPS 표시 후 출력

`q` 키를 누르면 종료합니다.

In [ ]:
fps = 0
st = time.time()
frames_to_count = 20
cnt = 0

try:
    while True:
        packet, _ = client_socket.recvfrom(BUFF_SIZE)

        # 서버에서 보낸 base64 문자열을 원본 바이트로 복원
        data = base64.b64decode(packet, " /")

        # OpenCV 디코딩을 위해 바이트 -> np.uint8 배열 변환
        npdata = np.frombuffer(data, dtype=np.uint8)
        frame = cv2.imdecode(npdata, 1)

        if frame is None:
            continue

        frame = cv2.putText(
            frame,
            f"FPS: {fps}",
            (10, 40),
            cv2.FONT_HERSHEY_SCRIPT_SIMPLEX,
            0.7,
            (0, 0, 255),
            2,
        )

        cv2.imshow("RECEIVING", frame)

        key = cv2.waitKey(1) & 0xFF
        if key == ord("q"):
            break

        if cnt == frames_to_count:
            now = time.time()
            elapsed = now - st
            if elapsed > 0:
                fps = round(frames_to_count / elapsed)
            st = now
            cnt = 0

        cnt += 1

finally:
    client_socket.close()
    cv2.destroyAllWindows()
    print("Client socket closed")

## 6) 트러블슈팅 체크리스트

- 서버와 클라이언트가 같은 네트워크 대역인지 확인
- `HOST_IP`, `PORT` 값이 서버 설정과 일치하는지 확인
- 방화벽에서 UDP 포트 차단 여부 확인
- 영상이 안 보이면 서버의 인코딩/전송 포맷(base64 + 이미지 코덱) 확인